In [28]:
#Importação das bibliotecas
import pandas as pd
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import urllib

In [68]:
#Escolhe o tipo de agendamento para avisar
while True:
    escolha = input('Escolha o tipo de aviso:\n<1>Exames\n<2>Consultas\n') 
    match escolha:
        case '1': #Mensagens para Exames
            df_aviso = pd.read_excel('exames.xlsx') #cria um dataframe com os exames
            df_aviso.loc[df_aviso['Controle'].isna(),'Controle'] = ''
            break
        case '2': #Mensagens para Consultas
            df_aviso = pd.read_excel('consultas.xlsx') #cria um dataframe com as consultas
            try: #Ajusta os campos caso venham no formato datetime
                df_aviso['Data'] = df_aviso['Data'].dt.strftime('%d/%m/%Y') #Ajusta a data
                df_aviso['Hora'] = df_aviso['Hora'].apply(lambda x: x.strftime('%H:%M')) #Ajusta a hora
            except Exception:
                pass #Ignora caso os campos já estejam no formato texto
            finally:
                df_aviso.loc[df_aviso['Controle'].isna(),'Controle'] = ''
                
            break
        case _:
            print('Escolha errada, tente novamente.')

In [30]:
def login_whats(driver): #função para realizar o login no whatsapp
    print('Verificando login.\n')
    while True:
        try:
            campo_busca = driver.find_element(By.XPATH,'//*[@id="side"]/div[1]/div/div[2]/div[2]/div/div[1]')
            print("Whatsapp logado.")
            break #whatsapp já está logado
        except NoSuchElementException:
            sleep(5) #tempo para aguardar o login
    sleep(3) #tempo para carregar a página após o login

In [31]:
def cria_mensagem(escolha,df,i):
    match escolha:
        case '1':
            mensagem = f'''Olá, Sr(a) *{df.loc[i,'Nome']}*
Estamos entrando em contato para comunicar o agendamento de: 
*{df.loc[i,'Procedimento']}*
*{df.loc[i,'Data/Hora']}*

Digite *1* Para *CONFIRMAR*
Digite *2* Para *REMARCAR* (Justifique o motivo)
Digite *3* Para *EXCLUIR* (Justifique o motivo)

at.te
Acolhe-PG
'''
        case '2':
            if df.loc[i,'Tipo Consulta'] == 'N':
                tipo_consulta = 'Consulta'
            else:
                tipo_consulta = 'Retorno'
            mensagem = f'''Olá, Sr(a) *{df.loc[i,'Nome']}*
Estamos entrando em contato para comunicar o agendamento de *{tipo_consulta}*: 
*{df.loc[i,'Procedimento']}* 
*{df.loc[i,'Profissional']}*
*{df.loc[i,'Data']}*
*{df.loc[i,'Hora']}*

Digite *1* Para *CONFIRMAR*
Digite *2* Para *REMARCAR* (Justifique o motivo)
Digite *3* Para *EXCLUIR* (Justifique o motivo)

at.te
Acolhe-PG
'''
    return mensagem

In [ ]:
#Abre o navegador
driver = webdriver.Chrome()
driver.get('https://web.whatsapp.com/')
#Loop para logar o captcha
login_whats(driver) 

In [ ]:
qtd_avisos = df_aviso.loc[df_aviso['Controle']!='Enviado','Controle'].count() #contabiliza a quantidade de envios que serão realizados
try:
        #Envia as mensagens para cada paciente na listagem
        for y,i in enumerate(df_aviso.loc[df_aviso['Controle']!='Enviado','Controle'].index): #percorre apenas os indices que serão utilizados para o contador
                print(f'Enviando mensagens, aguarde! Progresso: {y+1}/{qtd_avisos}')
                if df_aviso.loc[i,'Controle'] == 'Enviado': 
                        continue #pula a repetição caso já tenha enviado a mensagem
                else:
                        mensagem = cria_mensagem(escolha,df_aviso,i)
                        contato = df_aviso.loc[i,'Telefone']
                        if type(contato) == float: #Garante que a aplicação continua funcionando com valores nulos
                                df_aviso.loc[i,'Controle'] = 'Sem sucesso'
                                continue
                        texto = urllib.parse.quote(mensagem)
                        try:
                                driver.get(f'https://web.whatsapp.com/send/?phone=55${contato}&text={texto}')
                                
                                while len(driver.find_elements(By.ID,'side')) < 1:
                                        sleep(1)
                                sleep(3)
                                driver.find_element(By.XPATH,'//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[1]/div/div[1]/p').send_keys(Keys.ENTER)
                                df_aviso.loc[i,'Controle'] = 'Enviado'
                                sleep(5)
                                
                        except NoSuchElementException: #Exceção para quando não existe whatsapp cadastrado no número
                                df_aviso.loc[i,'Controle'] = 'Sem sucesso'
                                continue
                        except Exception as e:
                                print(f'Erro {type(e)} ao enviar para {contato}.') #detecta outros possíveis erros para posterior tratamento
                                continue
                        finally:
                            match escolha: #atualiza a planilha de acordo com o aviso realizado
                                case '1':
                                        df_aviso.to_excel('exames.xlsx',index=False)
                                case '2':
                                        df_aviso.to_excel('consultas.xlsx',index=False)     
        
                  
finally:
    print('Fim da aplicação.\nProgresso salvo na planilha.')
    driver.close()   
